# Gold — Foundry-ready event casefiles
Builds Gold casefiles and a flattened table for dashboards.

**Note**: evidence frame numbers are snapped to the nearest extracted frame number when sampling every N frames.

**Cell 1: Config**

In [7]:
from pyspark.sql import functions as F
from pyspark.sql import Window
import datetime, uuid

# Evidence settings
EVIDENCE_MAX_FRAMES_ANOM = 5   # max evidence frames per anomaly event
EVIDENCE_MAX_FRAMES_NORM = 3   # max evidence frames per normal event

# How far around the midpoint to search for normal evidence frames (ms)
MIDPOINT_TOL_MS = 1500

# New run id every execution
RUN_ID = "run_" + datetime.datetime.utcnow().strftime("%Y%m%d_%H%M%S") + "_" + uuid.uuid4().hex[:6]
SCHEMA_VERSION = "1.0"

print("RUN_ID =", RUN_ID)

StatementMeta(, 030436fc-8cb8-4ee9-b23d-9d74af67d8b2, 9, Finished, Available, Finished)

RUN_ID = run_20260109_153108_c93953


**Cell 2: Load Silver tables and normalize types**

In [8]:
events = spark.table("silver_event_windows") \
    .withColumn("t_start_ms", F.col("t_start_ms").cast("long")) \
    .withColumn("t_end_ms", F.col("t_end_ms").cast("long")) \
    .withColumn("label_prior", F.col("label_prior").cast("int"))

frames = spark.table("silver_frame_index") \
    .withColumn("ts_ms", F.col("ts_ms").cast("long")) \
    .withColumn("frame_number", F.col("frame_number").cast("int"))

labels = spark.table("silver_label_frame_index") \
    .withColumn("ts_ms", F.col("ts_ms").cast("long")) \
    .withColumn("frame_number", F.col("frame_number").cast("int")) \
    .withColumn("label_prior", F.col("label_prior").cast("int"))

print("events:", events.count())
print("frames:", frames.count())
print("labels:", labels.count())
display(events.groupBy("label_prior").count())

StatementMeta(, 030436fc-8cb8-4ee9-b23d-9d74af67d8b2, 10, Finished, Available, Finished)

events: 108
frames: 4713
labels: 2991


SynapseWidget(Synapse.DataFrame, cf5d5d9f-1adb-465d-a26b-76ea8455c0ec)

**Cell 3: Build anomaly evidence frames (label-driven)**

This produces evidence frames by taking anomaly-labeled frames inside each window and snapping to the nearest extracted frame (by frame_number).

In [9]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Join anomaly labels to event windows (only anomalies)
anom_events = events.filter(F.col("label_prior") == 1)

anom_labels_in_window = (labels.filter(F.col("label_prior") == 1)
    .join(anom_events.select("event_id","video_id","t_start_ms","t_end_ms"), on="video_id", how="inner")
    .where((F.col("ts_ms") >= F.col("t_start_ms")) & (F.col("ts_ms") < F.col("t_end_ms")))
    .select(
        "event_id",
        "video_id",
        F.col("ts_ms").alias("label_ts_ms"),
        F.col("frame_number").alias("label_frame_number"),
        "label_path"
    )
)

# Rename frame fields so nothing collides
frames_r = frames.select(
    "video_id",
    F.col("frame_number").alias("frame_number"),
    F.col("ts_ms").alias("frame_ts_ms"),
    "frame_path"
)

# For each label frame in a window, find the nearest extracted frame in that video (by frame_number)
cand = (anom_labels_in_window
    .join(frames_r, on="video_id", how="inner")
    .withColumn("frame_diff", F.abs(F.col("frame_number") - F.col("label_frame_number")))
)

w = Window.partitionBy("event_id","video_id","label_frame_number").orderBy(F.col("frame_diff").asc())

nearest = (cand
    .withColumn("rn", F.row_number().over(w))
    .filter(F.col("rn") == 1)
    .select(
        "event_id",
        "video_id",
        F.col("frame_number").alias("evidence_frame_number"),
        F.col("frame_ts_ms").alias("evidence_ts_ms"),
        "frame_path"
    )
)

# Limit to N evidence frames per event (earliest in time)
w2 = Window.partitionBy("event_id","video_id").orderBy(F.col("evidence_ts_ms").asc())

nearest_limited = (nearest
    .withColumn("rn2", F.row_number().over(w2))
    .filter(F.col("rn2") <= F.lit(EVIDENCE_MAX_FRAMES_ANOM))
)

anom_evidence_json = (nearest_limited
    .groupBy("event_id","video_id")
    .agg(F.to_json(F.collect_list(
        F.struct(
            F.col("evidence_frame_number").alias("frame_number"),
            F.col("evidence_ts_ms").alias("ts_ms"),
            F.col("frame_path").alias("frame_path")
        )
    )).alias("evidence_frames_json"))
)

display(anom_evidence_json.limit(10))
print("anom evidence rows:", anom_evidence_json.count())

StatementMeta(, 030436fc-8cb8-4ee9-b23d-9d74af67d8b2, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b185580d-766a-45b7-962a-3d4946a1bb9d)

anom evidence rows: 18


**Cell 4 — Build normal evidence frames (midpoint frames)**

For normal events, we pick frames near the midpoint of the window.

In [10]:
norm_events = events.filter(F.col("label_prior") == 0) \
    .withColumn("mid_ms", ((F.col("t_start_ms") + F.col("t_end_ms"))/2).cast("long"))

# Candidate frames within +/- MIDPOINT_TOL_MS of midpoint
norm_cands = (norm_events.select("event_id","video_id","t_start_ms","t_end_ms","mid_ms")
    .join(frames.select("video_id","frame_number","ts_ms","frame_path"), on="video_id", how="inner")
    .withColumn("dt", F.abs(F.col("ts_ms") - F.col("mid_ms")))
    .where(F.col("dt") <= F.lit(MIDPOINT_TOL_MS))
)

# Choose top K closest frames per event
w = Window.partitionBy("event_id","video_id").orderBy(F.col("dt").asc(), F.col("ts_ms").asc())

norm_nearest = (norm_cands
    .withColumn("rn", F.row_number().over(w))
    .filter(F.col("rn") <= F.lit(EVIDENCE_MAX_FRAMES_NORM))
    .select("event_id","video_id","frame_number","ts_ms","frame_path")
)

norm_evidence_json = (norm_nearest
    .groupBy("event_id","video_id")
    .agg(F.to_json(F.collect_list(
        F.struct(
            F.col("frame_number").alias("frame_number"),
            F.col("ts_ms").alias("ts_ms"),
            F.col("frame_path").alias("frame_path")
        )
    )).alias("evidence_frames_json"))
)

display(norm_evidence_json.limit(10))
print("norm evidence rows:", norm_evidence_json.count())

StatementMeta(, 030436fc-8cb8-4ee9-b23d-9d74af67d8b2, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 006d3f82-3953-4623-9e50-a5f3de81ac4c)

norm evidence rows: 90


**Cell 5: Combine evidence for all events**

In [11]:
evidence = (events.select("event_id","video_id")
    .join(anom_evidence_json, on=["event_id","video_id"], how="left")
    .join(norm_evidence_json.withColumnRenamed("evidence_frames_json","evidence_frames_json_norm"),
          on=["event_id","video_id"], how="left")
    .withColumn(
        "evidence_frames_json",
        F.coalesce(F.col("evidence_frames_json"), F.col("evidence_frames_json_norm"))
    )
    .drop("evidence_frames_json_norm")
)

display(evidence.limit(20))

StatementMeta(, 030436fc-8cb8-4ee9-b23d-9d74af67d8b2, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 17d5eec1-d3fc-4885-aca1-a8b5ec799378)

**Cell 6: Join PLC features**

In [12]:
plc_features = None
try:
    plc_features = spark.table("silver_plc_event_summary") \
        .select("event_id","video_id","plc_features_json")
    print("Loaded silver_plc_event_summary")
except Exception as e:
    print("silver_plc_event_summary not found/failed to load; continuing without PLC features:", e)

# If not available, create null column
if plc_features is None:
    plc_features = events.select("event_id","video_id").withColumn("plc_features_json", F.lit(None).cast("string"))

StatementMeta(, 030436fc-8cb8-4ee9-b23d-9d74af67d8b2, 14, Finished, Available, Finished)

Loaded silver_plc_event_summary


**Cell 7: Build gold_conveyor_event_fact**

In [13]:
gold = (events
    .join(evidence, on=["event_id","video_id"], how="left")
    .join(plc_features, on=["event_id","video_id"], how="left")
    .withColumn("run_id", F.lit(RUN_ID))
    .withColumn("schema_version", F.lit(SCHEMA_VERSION))
    .select(
        "run_id",
        "schema_version",
        "event_id",
        "video_id",
        "t_start_ms",
        "t_end_ms",
        "label_prior",
        "plc_features_json",
        "evidence_frames_json"
    )
)

# Replace only this run_id (safe if you rerun)
(gold.write
  .mode("overwrite")
  .format("delta")
  .option("replaceWhere", f"run_id = '{RUN_ID}'")
  .saveAsTable("gold_conveyor_event_fact")
)

display(gold.orderBy("label_prior", "video_id", "t_start_ms").limit(50))
print("Wrote: gold_conveyor_event_fact | rows:", gold.count())

StatementMeta(, 030436fc-8cb8-4ee9-b23d-9d74af67d8b2, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6b2de55d-1592-40ef-87a1-265743ff951d)

Wrote: gold_conveyor_event_fact | rows: 108


**Cell 8: Create gold_conveyor_event_fact_flat (flatten PLC features)**

If your silver_plc_event_summary already has flattened columns (like temp_mean/temp_max…), you can join those instead. But if you only have JSON, we’ll still write a flat table with the JSON column plus a couple common extracted fields.

This version writes a “flat” table that is still dashboard-friendly:

In [14]:
# Try to extract common fields from plc_features_json if present
# (If your JSON schema differs, it's okay—these will just be null.)
flat = (gold
    .withColumn("temp_mean", F.get_json_object(F.col("plc_features_json"), "$.temp_mean").cast("double"))
    .withColumn("temp_max",  F.get_json_object(F.col("plc_features_json"), "$.temp_max").cast("double"))
    .withColumn("motor_current_max", F.get_json_object(F.col("plc_features_json"), "$.motor_current_max").cast("double"))
)

(flat.write
  .mode("overwrite")
  .format("delta")
  .option("replaceWhere", f"run_id = '{RUN_ID}'")
  .saveAsTable("gold_conveyor_event_fact_flat")
)

display(flat.limit(50))
print("Wrote: gold_conveyor_event_fact_flat | rows:", flat.count())

display(flat.groupBy("label_prior").count())
display(flat.select("run_id").distinct())

StatementMeta(, 030436fc-8cb8-4ee9-b23d-9d74af67d8b2, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e83c3b26-ca97-4c3f-9aca-6999c0d72f72)

Wrote: gold_conveyor_event_fact_flat | rows: 108


SynapseWidget(Synapse.DataFrame, 29579aeb-0cfa-4fdf-a3d6-788d53f056e5)

SynapseWidget(Synapse.DataFrame, 8e186bc2-86dd-4cf1-a890-3489b0f557b3)

**Cell 9: Sanity queries**

In [15]:
# Latest run summary (since you may only have one run at first)
latest_run = flat.select("run_id").distinct().orderBy(F.col("run_id").desc()).limit(1).collect()[0]["run_id"]
print("latest_run =", latest_run)

display(flat.filter(F.col("run_id") == latest_run).groupBy("label_prior").count())

print("total events latest run:", flat.filter(F.col("run_id") == latest_run).count())
print("anomalies latest run:", flat.filter((F.col("run_id") == latest_run) & (F.col("label_prior") == 1)).count())

StatementMeta(, 030436fc-8cb8-4ee9-b23d-9d74af67d8b2, 17, Finished, Available, Finished)

latest_run = run_20260109_153108_c93953


SynapseWidget(Synapse.DataFrame, 51c01d11-f069-4bdd-bd56-02772f14f6ed)

total events latest run: 108
anomalies latest run: 18
